In [1]:
#!/usr/bin/env conda run -n newenv3 python
import sys
print(sys.executable)    ### C:\ProgramData\Anaconda3_1\envs\newenv3\python.exe
#!{sys.executable} -m pip install tables              ##  df.to_hdf

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
!jupyter kernelspec list
# pip install xlrd
# python -m pip install --user numpy scipy matplotlib ipython jupyter pandas sympy nose

C:\ProgramData\Anaconda3_1\envs\newenv3\python.exe


Available kernels:
  python3    C:\ProgramData\Anaconda3_1\envs\newenv3\share\jupyter\kernels\python3


In [2]:
import matplotlib.pyplot as plt
from descartes import PolygonPatch
import math
from ipyleaflet import Map, GeoJSON
import geojson
import numpy as np
#from mpl_toolkits.basemap import Basemap
from ipyleaflet import Map
import mpld3
from dbfread import DBF
mpld3.enable_notebook()
#to import the basemap library give the direct path to the library
# import os
# os.environ["PROJ_LIB"]="C:\\Users\\Anaconda3\\Library\\share"
import pandas as pd
from simpledbf import Dbf5
import math
from Finddistribution import *
import numpy as np

from scipy.stats import expon
import matplotlib.pyplot as plt

import os
import types

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon


import plotly.graph_objects as go
from scipy.interpolate import griddata
from pyproj import Proj
import plotly.express as px

import random
from tqdm import tqdm

import sys
from tqdm import trange
import seaborn as sns

SQLalchemy is not installed. No support for SQL output.


In [4]:
df_zo = pd.read_csv('df_zo.csv', sep=';', index_col=0)
df_zo_100 = pd.read_csv('df_zo_100.csv', sep=';', index_col=0)
df = pd.read_csv('df.csv', sep=';', index_col=0)
df0 = pd.read_csv('df0.csv', sep=';', index_col=0)

In [23]:
df.columns

Index(['IGE', 'Litho', 'POINT_X', 'POINT_Y', 'h_quat', 'Tipo', 'h_rilevato',
       'h_perm', 'progressiva', 'vtrackupx', 'vtrackdownx', 'vtrackup2000x',
       'vtrackdown2000x', 'vtrackupy', 'vtrackdowny', 'vtrackup2000y',
       'vtrackdown2000y'],
      dtype='object')

In [22]:
# import plotly.graph_objects as go

x = df.POINT_X
y = df.POINT_Y

df0.dropna(inplace = True)
sizeref = 2.*max(df0['dave'])/(300)
# f = px.scatter(df0, x='long', y='lat', color='sector2000_50_str', size='dave')
# # senza colori per settore
f = px.scatter(df0, x='long', y='lat', size='dave', hover_data=['sector2000_50','dbase'])

sizeref = 2.*max(df_zo.dbase)/300
f = px.scatter(df_zo, x='long', y='lat', size='dbase', hover_data=['sector2000_50','Pds'])


f.update_traces(marker=dict(
    sizemode='area', sizeref=sizeref, line_width=2))

# f.add_trace(go.Scatter(x=df0.long, y=df0.lat, mode='markers', marker_size=df0.dave, marker=dict(
#     sizemode='area', sizeref=sizeref, line_width=2)))

# senza spessore punti
f.add_trace(go.Scatter(x=df_zo[(df_zo['dbase'] > 0) & (df_zo.Pds >= 0.01)]['long'], y=df_zo[(
    df_zo['dbase'] > 0) & (df_zo.Pds >= 0.01)]['lat'], mode='markers'))

# con spessore punti
# f.add_trace(go.Scatter(x=df_zo[df_zo['dbase'] > 0]['long'], y=df_zo[df_zo['dbase'] > 0]['lat'], mode='markers', marker_size=sizeref*10))

f.add_trace(go.Scatter(x=x, y=y))


f.add_trace(go.Scatter(x=df.vtrackupx, y=df.vtrackupy))
f.add_trace(go.Scatter(x=df.vtrackdownx, y=df.vtrackdowny))

f.add_trace(go.Scatter(x=df.vtrackup2000x, y=df.vtrackup2000y))
f.add_trace(go.Scatter(x=df.vtrackdown2000x, y=df.vtrackdown2000y))



f.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
)


f.update_layout(
    autosize=False,
    width=1100,
    height=800,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=4
    ), legend = dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

f.show()

In [20]:
hh=10

# cerco l'area con più karst
# e quella con karst minimo (il minimo viene definito qui)
n_karst_soglia = 70
# numero max
n_max_settore = int(df_g50.n_karst.max())
# settore con numero max
settore_nmax= df_g50[df_g50.n_karst == df_g50.n_karst.max()].sector2000_50.tolist()[0]
# settore con numero karst basso ma maggiore di 5:
settore_nmin= df_g50[df_g50.n_karst >= n_karst_soglia].sort_values("n_karst").sector2000_50.tolist()[0]

# se non metto nulla negli attributi della dex usa il max
if hh  == 'none':
    h = settore_nmax

elif df_g50[df_g50.sector2000_50==hh].n_karst.tolist()[0] < n_karst_soglia:
    h = settore_nmin

else:
    h = hh

zo_dave = df0[(df0.sector2000_50==h)&(df0.era == "Giovane")].dbase
# per aumentare la stima del numero considero anche quelli antichi
#zo_dave = df0[df0.sector2000_50==h].dbase

# TO DO TRY WITH st.truncexp
params = st.expon.fit(zo_dave, floc=0)

scale = params[1]/2

zo_perc5 = int(np.percentile(zo_dave, 5))
zo_perc95 = int(np.percentile(zo_dave, 95))
zo_karstnum = zo_dave.shape[0]

zo_perc5, zo_perc95, zo_karstnum

# calcolo le proporzioni tra i karst sotto il 5p
# 1. zo_karstnum_simul è il numero su 1000 dei karst individuati a rilievo
# 2. zo_nbelow5p_simul è la stima su 1000 dei karst non individuati
# zo_tot è il totale della popolazione (tolto il frattile superiore al 95% che viene considerato un errore)
# zo_tot_prop è la proporzione tra il totale della simulazione e 1+2
# zo_pop_prop è la proporzione tra il totale della popolazione (compreso frattile >95%) e 1+2

size = 1000
r = expon.rvs(scale=scale, size=size)

zo_karstnum_simul = r[(zo_perc5<r)&(r<zo_perc95)].shape[0]
zo_nbelow5p_simul = r[r<zo_perc5].shape[0]
zo_tot = zo_karstnum_simul + zo_nbelow5p_simul
zo_tot_prop = zo_tot/zo_karstnum_simul
zo_pop_prop = size/zo_karstnum_simul

zo_karstnum_simul, zo_nbelow5p_simul, zo_tot, zo_tot_prop, zo_pop_prop

zo_nbelow5p = int(round(zo_karstnum/zo_karstnum_simul*zo_nbelow5p_simul,0))
zo_nbelow5p

# the number of data to obtain 27 karst less than 6m is
# 37/538*1000




zo_karstnum_pop = int(round(zo_karstnum/zo_karstnum_simul*r.shape[0],0))
zo_karstnum_pop

# zo_karstnum_pop = sono quelli totali della simulazione
# zo_karstnum = sono quelli effettivi individuati 95p e non individuati)

zo_karstnum_pop = int(round(zo_pop_prop *zo_karstnum, 0))

return zo_nbelow5p, zo_karstnum_pop, zo_karstnum, settore_nmax, scale

NameError: name 'df_g50' is not defined

In [25]:
df_zo_100.columns

Index(['long', 'lat', 'dave', 'sector2000_50', 'sector2000_50_str', 'h_perm',
       'h_quat', 'IGE', 'dbase', 'Tipo', 'h_rilevato', 'Pds', 'check_100',
       'epoch', 'ril_ksmall', 'ril_klarge', 'tri_ksmall', 'tri_klarge',
       'via_ksmall', 'via_klarge', 'pas_ksmall', 'pas_klarge'],
      dtype='object')